In [9]:
!unzip sss.zip


Archive:  sss.zip
   creating: New folder/Clean/
  inflating: New folder/Clean/1.txt  
  inflating: New folder/Clean/10.js  
  inflating: New folder/Clean/11.js  
  inflating: New folder/Clean/12.js  
  inflating: New folder/Clean/13.js  
  inflating: New folder/Clean/14.js  
  inflating: New folder/Clean/15.js  
  inflating: New folder/Clean/16.js  
  inflating: New folder/Clean/17.js  
  inflating: New folder/Clean/18.js  
  inflating: New folder/Clean/19.js  
  inflating: New folder/Clean/2.txt  
  inflating: New folder/Clean/20.js  
  inflating: New folder/Clean/21.js  
  inflating: New folder/Clean/22.js  
  inflating: New folder/Clean/23.js  
  inflating: New folder/Clean/24.js  
  inflating: New folder/Clean/25.js  
  inflating: New folder/Clean/26.js  
  inflating: New folder/Clean/27.js  
  inflating: New folder/Clean/28.js  
  inflating: New folder/Clean/29.js  
  inflating: New folder/Clean/3.txt  
  inflating: New folder/Clean/30.js  
  inflating: New folder/Clean/31.js  
 

In [10]:
cd sss


/content/sss


In [12]:
import os

clean_files = []
dirty_files = []

for filename in os.listdir('./Clean'):
    with open(os.path.join('./Clean', filename), 'r') as f:
        clean_files.append(f.read())

for filename in os.listdir('./Dirty'):
    with open(os.path.join('./Dirty', filename), 'r') as f:
        dirty_files.append(f.read())


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

clean_tokens = []
dirty_tokens = []

for file_contents in clean_files:
    tokens = word_tokenize(file_contents)
    clean_tokens.append(tokens)

for file_contents in dirty_files:
    tokens = word_tokenize(file_contents)
    dirty_tokens.append(tokens)


In [16]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np

tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_tokens + dirty_tokens)

clean_sequences = tokenizer.texts_to_sequences(clean_tokens)
dirty_sequences = tokenizer.texts_to_sequences(dirty_tokens)

max_length = max(len(seq) for seq in clean_sequences + dirty_sequences)
clean_sequences_padded = pad_sequences(clean_sequences, maxlen=max_length)
dirty_sequences_padded = pad_sequences(dirty_sequences, maxlen=max_length)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length))
model.add(LSTM(units=64))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X = np.concatenate([clean_sequences_padded, dirty_sequences_padded], axis=0)
y = np.concatenate([np.ones(len(clean_sequences_padded)), np.zeros(len(dirty_sequences_padded))], axis=0)

model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
2/2 [==============================] - 7s 2s/step - loss: 0.6819 - accuracy: 0.8906 - val_loss: 0.7049 - val_accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 2s 1s/step - loss: 0.6627 - accuracy: 0.8750 - val_loss: 0.7204 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 2s 1s/step - loss: 0.6408 - accuracy: 0.8750 - val_loss: 0.7420 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 2s 1s/step - loss: 0.6100 - accuracy: 0.8750 - val_loss: 0.7762 - val_accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 3s 2s/step - loss: 0.5651 - accuracy: 0.8750 - val_loss: 0.8402 - val_accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 5s 3s/step - loss: 0.4916 - accuracy: 0.8750 - val_loss: 1.0185 - val_accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 2s 1s/step - loss: 0.3587 - accuracy: 0.8750 - val_loss: 1.8664 - val_accuracy: 0.0000e+00
Epoch 

In [17]:
model.save('my_model.h5')
